In [1]:
!pip install face-recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import face_recognition
import cv2
import numpy as np

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image

In [3]:
ref_image = face_recognition.load_image_file("ref.jpeg")
ref_face_encoding = face_recognition.face_encodings(ref_image)[0]

known_face_encodings = [
    ref_face_encoding
]
known_face_names = [
    "Senura"
]

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

In [4]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [6]:
def verify():
  target = cv2.imread('photo.jpg')
  small_frame = cv2.resize(target, (0, 0), fx=0.25, fy=0.25)

  rgb_small_frame = small_frame[:, :, ::-1]

  face_locations = face_recognition.face_locations(rgb_small_frame)
  face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

  face_names = []
  name = ""
  for face_encoding in face_encodings:
      matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

      if True in matches:
          first_match_index = matches.index(True)
          name = known_face_names[first_match_index]

      face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
      best_match_index = np.argmin(face_distances)
      if matches[best_match_index]:
          name = known_face_names[best_match_index]
  if name == "":
      return False
  else:
      return True

In [7]:
if verify():
  print("Verified")
else:
  print("Not Verified")

Not Verified
